In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

## Первым делом, загрузим и изучим предоставленные датасеты

In [18]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [19]:
train_data.head(5)

,label,email
0,spam,attn i am bulawa mulete jr the son of mr steve...
1,ham,That depends. How would you like to be treated...
2,ham,"Wen u miss someone, the person is definitely s..."
3,spam,Win the newest “Harry Potter and the Order of ...
4,ham,Where are you lover ? I need you ...


In [20]:
test_data.head(5)

,label,email
0,ham,Why you keeping me away like this
1,ham,neale pickett if you can spare the memory you...
2,ham,I think u have the wrong number.
3,ham,* Was thinking about chuckin ur red green n bl...
4,ham,i am trying to manage the email for a domain w...


In [22]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5863 entries, 0 to 5862
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5844 non-null   object
 1   email   5844 non-null   object
dtypes: object(2)
memory usage: 91.7+ KB


In [23]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2889 entries, 0 to 2888
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   2889 non-null   object
 1   email   2889 non-null   object
dtypes: object(2)
memory usage: 45.3+ KB


## Удалим пропущенные значения и аномальные метки

In [24]:
train_data = train_data.dropna().reset_index(drop=True)
train_data = train_data[train_data['label'].isin(['ham', 'spam'])].reset_index(drop=True)

## Разделим обучающую выборку на обучающую и валидационную и после векторизируем значения в столбце 'email'. Тексты векторизуются с помощью CountVectorizer

In [25]:
X_train, X_val, y_train, y_val = train_test_split(train_data['email'], train_data['label'], test_size=0.2, random_state=42)


vectorizer = CountVectorizer(stop_words='english')

X_train_vect = vectorizer.fit_transform(X_train)
X_val_vect = vectorizer.transform(X_val)

test_vect = vectorizer.transform(test_data['email'])

## Для обучения реализуем нативный байесовский алгоритм, так как он подходит для классификации текстов

In [26]:
model = MultinomialNB()
model.fit(X_train_vect, y_train)

MultinomialNB()

## Далее оценим модель на валидационной выборке, и выводем точность по метрикам, исаользующихся при оценке задачи классификации (f-score, precision, recall)

In [27]:
val_predictions = model.predict(X_val_vect)
val_accuracy = accuracy_score(y_val, val_predictions)
val_report = classification_report(y_val, val_predictions)

print("Validation Accuracy:", val_accuracy)
print("Classification Report:\n", val_report)

Validation Accuracy: 0.959656652360515
Classification Report:
               precision    recall  f1-score   support

         ham       0.98      0.97      0.98       967
        spam       0.86      0.91      0.88       198

    accuracy                           0.96      1165
   macro avg       0.92      0.94      0.93      1165
weighted avg       0.96      0.96      0.96      1165



## Судя по метрикам, результат получился удовлетворительным, осталось только предсказать значения на тестовой выборке и вывести точность (accuracy)

In [28]:
test_data['predicted_label'] = model.predict(test_vect)

In [29]:
test_accuracy = accuracy_score(test_data['label'], test_data['predicted_label'])

print(test_accuracy)

0.9691934925579785


## Точность получилась выше 95% Считаю этот результат очень хорошим :)
PS Не знал нужно ли было прямо супер подробно обо всем рассказывать(в частности как работает CountVectorizer, там стоп-слова и тд., что такое Naive Bayes). Думаю, все это подробно можно спросить на тех. собеседовании )
